# Model V1

**Summary**: Changed model to be implemented in PyTorch (doesn't include LSTM). Uses a definitely working implementation of the Policy Gradient algorithm. Environment observation returns differences in prices rather than entire prices.

-------------------------
## The Model
Before, the model encountered problems where it was constantly buying in the tensorflow model. To reduce the risk of an error on my part, I have re-written a simpler model in PyTorch. PyTorch uses dynamic construction of the computation graph so it will have better support later on when implementing a LSTM. The current shape of the model is as follows: Input (5) -> Layer 1 (64) -> Layer 2 (32) -> Layer 3 (64) -> Layer 4 (16) -> Layer 5 (3). As a result of these changes, I expect the accuracy of the model to get above a 65% for the current implementation of the archeticture to be labeled as a success. The model has also been given the function `generate_action_call()` which allows for it to create the action calls to the environment. The `amount` variable will later be the subject to changing as it controls how much stock the agent wishes to buy or sell. `torch.set_default_tensor_type('torch.cuda.FloatTensor')` was used to ensure that GPU would be used by PyTorch.

**TODO**
* Determine How Much To Buy
* Create Testing Loop
-----------------------------
## The Environment
Only a couple of changes have been made to the environment. They include ...
* Holding_Rate changed to 0.1 -> 0.01
* State holds records of the current days changes in prices. The returned value of today displayed to the model is not the raw number, but rather **today - yesterday** for all 5 features (Volume, Open, High, Low, Close)

**Posible Changes** in later iterations maybe be to include ...
* Include Algo Trading Ratios (Sharpe Ratio...) into the input into the model
------------------------------------------
## Training
The optimization occurs after every episode because the rewards and depreciation would be messed up if it occured after every episode. The entire loop was re-written with PyTorch in mind. Also, the agent will not always be reseted to the 0th day of the dataset because not reseting of the dataset occurs with exception to the `env.load_stock()` function.

**Hyperparameters Check**
* `n_epochs = 500` |  How many stocks will be iterated throught
* `n_batches = 10`  |  How many runs on a single stock in a row
* `learning_rate = 1e-4`  |  Amount of change to take when making changes to the model's weights
* `gamma = .95`  |  Decay rate of rewards as accordance with Policy Gradient spec
* `max_steps_per_episode = 90`  |  Maximum amount of days the agent can run before being terminated
* `running_reward = 10` |  Initial reward value for reward averager (Depreciated Overtime)
* `reward_threshold = 300`  |  Reward needed to obtain success on this model's run

# Model V2

## The Model
In this implementation, we created a 128 node LSTM to recieve that past 30 days of data so the model will be aple to make more informed decision on trajectory of a stock in addition to the price difference recordings made in the previous version. Additionally, a new hyperparameters has been introduced, dropout, to assist in the normalization of the training session. Observed was quite sporadic changes in the reward from epoch to epoch, so introducing dropout  will ensure that the overfitting is not present in future implementations (See _Hyperparameter Check_ to view the amount of dropout)
So, I had to re-write all of the model, but its working. Pretty much there were some issues where the weights were being turned into NaNs and only returning 0 for all of the values and I went ahead and re-wrote V2 so it actually worked. Now, its in one file, and pretty nice to look at I guess. All of the previously proposed architecture still stands. 

___Model___
* LSTM: (30, 5) -> 128
* Layer 1: 128 -> 64
* Dropout: .5
* Layer 2: 62 -> 32
* Dropout: .5
* Layer 3: 32 -> 64
* Dropout: .5
* Layer 4: 64 -> 16
* Layer 5: 16 -> 3


## The Environment
A recurring error found during training was the model constantly returning 0 during prediction phases. This was result of the model having weights being changed to NaNs, which was caused by some part of the code returning NaNs. I determined through experimentation that the source were NaNs in the training data originating from `df.dropna()` not being an inplace function. With this function solved, the entire training process was fixed. Also, the environment only returns the difference from day to day instead of the actual values (That difference is normalized before going through the LSTM becaus

## Training
The training step will also get rid of the `running_reward` implementation for tracking the accuracy of the model but rather  a sum_reward which is meaned at print time. Also, the new `reward_threshold` is based off of the 5% return of a bank is required by the model to be deemed _successful_.

**Hyperparameters Check**
* `n_epochs / n_stock = 5000` |  How many stocks will be iterated throught
* `n_batches / session_on_stock = 25`  |  How many runs on a single stock in a row
* `learning_rate = 1e-4`  |  Amount of change to take when making changes to the model's weights
* `gamma = .95`  |  Decay rate of rewards as accordance with Policy Gradient spec
* `max_steps_per_episode = 90`  |  Maximum amount of days the agent can run before being terminated
* `reward_threshold = 5250`  |  Reward needed to obtain success on this model's run (Based off of the 5% average annual return from banks)